In [1]:
import time

import numpy as np

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Softmax
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Accuracy, Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau

from HW_base import evaluate_build, focus_build
from HW_keras import hw_layer

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [4]:
x_min = np.min(x_train, axis=(1, 2), keepdims=True)
x_max = np.max(x_train, axis=(1, 2), keepdims=True)
x_train = (x_train - x_min) / (x_max - x_min)

In [5]:
x_min = np.min(x_test, axis=(1, 2), keepdims=True)
x_max = np.max(x_test, axis=(1, 2), keepdims=True)
x_test = (x_test - x_min) / (x_max - x_min)

In [6]:
evaluate_list = [evaluate_build(x_test[..., i], 32) for i in range(x_test.shape[-1])]
evaluate_focus_list = [focus_build(evaluate, 0.8) for evaluate in evaluate_list]

evaluate_num:  32,focus:0.8000: 32it [00:00, 2292.05it/s]
evaluate_num:  32,focus:0.8000: 32it [00:00, 1887.49it/s]
evaluate_num:  32,focus:0.8000: 32it [00:00, 2139.51it/s]


In [7]:
x = Input(shape=((32, 32, 3)))
y = x

y = hw_layer(evaluate_focus_list)(y)
y = Softmax(axis=-1)(y)

y = Conv2D(32, 3, 2, 'same')(y)
y = Conv2D(32, 1, 1, 'same')(y)
y = Softmax(axis=-1)(y*8)

y = Conv2D(32, 3, 2, 'same')(y)
y = Conv2D(32, 1, 1, 'same')(y)
y = Softmax(axis=-1)(y*8)

y = Conv2D(32, 3, 2, 'same')(y)
y = Conv2D(32, 1, 1, 'same')(y)
y = Softmax(axis=-1)(y*8)

y = Flatten()(y)
y = Dense(10)(y)
y = Activation('softmax')(y)

model = Model(x, y)
model.summary()

model.compile(loss=Huber(), optimizer=SGD(learning_rate=0.001), metrics=[AUC(), Accuracy()])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
hw_layer (hw_layer)          (None, 32, 32, 96)        0         
_________________________________________________________________
softmax (Softmax)            (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 32)        27680     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        1056      
_________________________________________________________________
tf.math.multiply (TFOpLambda (None, 16, 16, 32)        0         
_________________________________________________________________
softmax_1 (Softmax)          (None, 16, 16, 32)        0     

In [8]:
datagen = ImageDataGenerator(
    featurewise_center = False,  # 将整个数据集的均值设为0
    samplewise_center = False,  # 将每个样本的均值设为0
    featurewise_std_normalization = False,  # 将输入除以整个数据集的标准差
    samplewise_std_normalization = False,  # 将输入除以其标准差
    zca_whitening = False,  # 运用 ZCA 白化
    zca_epsilon = 1e-06,  # ZCA 白化的 epsilon值
    rotation_range = 0,  # 随机旋转图像范围 (角度, 0 to 180)
    width_shift_range = 0.1,  # 随机水平移动图像 (总宽度的百分比) 
    height_shift_range = 0.1,  # 随机垂直移动图像 (总高度的百分比)
    shear_range = 0.,  # 设置随机裁剪范围
    zoom_range = 0.,  # 设置随机放大范围
    channel_shift_range = 0.,  # 设置随机通道切换的范围
    fill_mode = 'nearest',  # 设置填充输入边界之外的点的模式
    cval = 0.,  # 在 fill_mode = "constant" 时使用的值
    horizontal_flip = True,  # 随机水平翻转图像
    vertical_flip = False,  # 随机垂直翻转图像
    rescale = None,  # 设置缩放因子 (在其他转换之前使用)
    preprocessing_function = None,  # 设置将应用于每一个输入的函数
    data_format = None,  # 图像数据格式，"channels_first" 或 "channels_last" 之一
    validation_split = 0.0)  # 保留用于验证的图像比例（严格在0和1之间）

# 计算特征标准化所需的计算量
# (如果应用 ZCA 白化，则为 std，mean和主成分).
datagen.fit(x_train)

In [9]:
model_name = time.strftime('CIFAR10_KERAS_HWNET%Y%m%d%H%M%S')

callback_list = [
    ReduceLROnPlateau(monitor='loss', factor=0.7,  patience=4, min_lr=1e-7, verbose=True),
    EarlyStopping(monitor='loss', patience=32),
    ModelCheckpoint('models/%s_best.h5'%model_name, monitor='loss', save_best_only=True, verbose=False),
    TensorBoard(log_dir='./Log/%s'%model_name)
]

model.fit(datagen.flow(x_train, y_train, batch_size=1000), epochs=200, validation_data=(x_test, y_test), callbacks=callback_list)

Epoch 1/200
50/50 [==============================] - 15s 244ms/step - loss: 0.0450 - auc: 0.5000 - accuracy: 0.0000e+00 - val_loss: 0.0450 - val_auc: 0.5000 - val_accuracy: 0.0000e+00
Epoch 2/200
50/50 [==============================] - 12s 230ms/step - loss: 0.0450 - auc: 0.5000 - accuracy: 0.0000e+00 - val_loss: 0.0450 - val_auc: 0.5000 - val_accuracy: 0.0000e+00
Epoch 3/200
50/50 [==============================] - 12s 230ms/step - loss: 0.0450 - auc: 0.5000 - accuracy: 0.0000e+00 - val_loss: 0.0450 - val_auc: 0.5000 - val_accuracy: 0.0000e+00
Epoch 4/200
50/50 [==============================] - 12s 231ms/step - loss: 0.0450 - auc: 0.5000 - accuracy: 0.0000e+00 - val_loss: 0.0450 - val_auc: 0.5000 - val_accuracy: 0.0000e+00
Epoch 5/200
50/50 [==============================] - 11s 227ms/step - loss: 0.0450 - auc: 0.5000 - accuracy: 0.0000e+00 - val_loss: 0.0450 - val_auc: 0.5000 - val_accuracy: 0.0000e+00

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0007000000332482159.

50/50 [==============================] - 11s 226ms/step - loss: 0.0450 - auc: 0.5000 - accuracy: 0.0000e+00 - val_loss: 0.0450 - val_auc: 0.5000 - val_accuracy: 0.0000e+00

Epoch 00041: ReduceLROnPlateau reducing learning rate to 2.8247522277524694e-05.
Epoch 42/200
50/50 [==============================] - 11s 228ms/step - loss: 0.0450 - auc: 0.5000 - accuracy: 0.0000e+00 - val_loss: 0.0450 - val_auc: 0.5000 - val_accuracy: 0.0000e+00
Epoch 43/200
50/50 [==============================] - 11s 226ms/step - loss: 0.0450 - auc: 0.5000 - accuracy: 0.0000e+00 - val_loss: 0.0450 - val_auc: 0.5000 - val_accuracy: 0.0000e+00
Epoch 44/200
50/50 [==============================] - 11s 226ms/step - loss: 0.0450 - auc: 0.5000 - accuracy: 0.0000e+00 - val_loss: 0.0450 - val_auc: 0.5000 - val_accuracy: 0.0000e+00
Epoch 45/200
50/50 [==============================] - 11s 228ms/step - loss: 0.0450 - auc: 0.5000 - accuracy: 0.0000e+00 - val_loss: 0.0450 - val_auc: 0.5000 - val_accuracy: 0.0000e+00

Epoch